In [19]:
!pip install pyspark==3.3.1

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


25/01/22 21:04:23 WARN Utils: Your hostname, ANDERSON resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/01/22 21:04:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/22 21:04:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.json("../../datalake/twitter_datascience")

In [5]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |--

In [7]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [8]:

df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{1468226301643403...|
|{8193148897894932...|
|{1317255963762634...|
|{1497937787181363...|
|{1468226301643403...|
|{1468226301643403...|
|{1220990287234969...|
|{1459140443954978...|
|{25204382, 159774...|
|{1011817655957893...|
|{9354468297302917...|
|{190097582, 15977...|
|{9354468297302917...|
|{142255858, 15977...|
|{1142424032794406...|
|{7101237361757839...|
|{2415478665, 1597...|
|{44894396, 159774...|
|{190097582, 15977...|
|{9354468297302917...|
+--------------------+
only showing top 20 rows



In [9]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [10]:
tweet_df.show(5)

+-------------------+-------------------+--------------------+-------------------+----------+-----------+-----------+-------------+--------------------+
|          author_id|    conversation_id|          created_at|                 id|like_count|quote_count|reply_count|retweet_count|                text|
+-------------------+-------------------+--------------------+-------------------+----------+-----------+-----------+-------------+--------------------+
|1468226301643403271|1597742169376579584|2022-11-29T23:59:...|1597742169376579584|         0|          0|          0|           98|RT @ratingology: ...|
| 819314889789493248|1597742113441341441|2022-11-29T23:59:...|1597742113441341441|         0|          0|          0|            2|RT @Datascience__...|
|1317255963762634752|1597742042284978176|2022-11-29T23:59:...|1597742042284978176|         0|          0|          0|            1|RT @genericgranol...|
|1497937787181363202|1597742024555274242|2022-11-29T23:59:...|1597742024555274242|

In [11]:
df.select(f.explode("includes.users")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- username: string (nullable = true)



In [12]:
df.select(f.explode("includes.users").alias("users")).printSchema()

root
 |-- users: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- username: string (nullable = true)



In [13]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [14]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [15]:
user_df.show(5)

+--------------------+-------------------+--------------------+--------------+
|          created_at|                 id|                name|      username|
+--------------------+-------------------+--------------------+--------------+
|2021-12-07T14:30:...|1468226301643403271|        Rakesh Kumar|      kasavsam|
|2017-01-11T22:47:...| 819314889789493248|Parody Aaron Sciv...|  Eintsein1980|
|2020-10-17T00:08:...|1317255963762634752|            Omua Mua|4constellation|
|2022-02-27T14:13:...|1497937787181363202|       MarkKelvin_44| MarkKelvin_44|
|2020-01-25T08:42:...|1220990287234969600|      micopia.online| micopiaonline|
+--------------------+-------------------+--------------------+--------------+
only showing top 5 rows



In [16]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')

In [17]:
user_df.coalesce(1).write.mode("overwrite").json('output/user')

In [18]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')